## Importando pacotes e inicializando *geemap*


In [ ]:
import os
import ee
import geemap

In [ ]:
geemap.ee_initialize()

A diversidade da paisagem é uma métrica composta pelos valores de Z da variedade de *landforms*, amplitude altitudinal, *wetland score* e diversidade de solos. Inicialmente, a diversidade da paisage assume os valores de variedade de *landforms*. Nos locais onde a amplitude de elevação é maior que variedade de *landforms*, a diversidade da paisagem é substituída pela média ponderada entre as duas variáveis, atribuindo peso dobrado para a variedade de *landforms*. Locais onde wetland score é maior que a diversidade da paisagem calculada anteriormente, os valores são substituídos por wetland score. Na localidades onde a diversidade de solo é maior que a diversidade da paisagem, os valores são substituídos pela média ponderada das variáveis naquela localidade.

## Base de Dados

Os dados para a diversidade da paisagem são os criados na etapa de cálculo dos *Z-scores*, descrito anteriormente.

## Códigos para a criação da diversidade da paisagem

Nossas análises foram rodadas no Google Earth Engine (Gorelick 2017), devido a demanda computacional do projeto, usando o pacote geemap (Wu 2020) em Python (Python Software Foundation 2023) como interface pela facilidade na documentação e reprodutividade das análises.

Nós importamos os dados de bioma para delimitar a região de análise e o raster do Brasil para que os mapas sejam cortados para o Brasil antes do cálculo da diversidadade da paisagem.


In [ ]:
# Importando o polígono de bioma para definir as 
# coordenadas máximas e mínimas do Brasil

bioma = ee.FeatureCollection("projects/ee-lucasljardim9/assets/Biome")

brasil_raster = ee.Image("projects/ee-lucasljardim9/assets/brasil_raster")

# ModeloDigital de Elevação para extrair a resolução
DEM = ee.Image("MERIT/DEM/v1_0_3")

# função para extrair as bordas dos polígonos
def func_cmp(feature):
    return feature.bounds() 

# Extraindo as coordenadas mínimas e máximas do Brasil
bioma_box = bioma.map(func_cmp).geometry().dissolve(**{'maxError': 1}).bounds()

# Resolução das análises
escala = DEM.projection().nominalScale()

Importamos os mapas e cortamos para o raster do Brasil.


In [ ]:
# Importando e cortando as imagens para o Brasil
landform_Z = (ee.Image('projects/ee-lucasljardim9/assets/Z_landform_variety')
              .updateMask(brasil_raster))

elevation_range_Z = (ee.Image('projects/ee-lucasljardim9/assets/Z_elevation_range')
                     .updateMask(brasil_raster))

wetland_score = (ee.Image('projects/ee-lucasljardim9/assets/Z_wetlands_score')
                 .updateMask(brasil_raster))

soil_diversity =  (ee.Image('projects/ee-lucasljardim9/assets/Z_soil_diversity')
                   .updateMask(brasil_raster))

Aqui começamos o cálculo da diversidade da paisagem, atribuindo à diversidade da paisagem os valores de variedade de *landforms*. Depois testamos os locais onde a amplitude de elevação é maior que a variedade de *landforms* e substituímos os valores.


In [ ]:
landscape_diversity = landform_Z

# Testando se o Z da amplitude de elevação é maior que o Z da variedade de landsforms
test_1 = elevation_range_Z.gt(landscape_diversity)

# Média ponderada entre variedade de landforms e amplitude de elevação
average_elevation = landform_Z.multiply(2).add(elevation_range_Z).divide(3)

# Substituindo os valores de variedade de landforms por amplitude de elevação 
# onde o segundo valor é maior que o primeiro
landscape_diversity = landscape_diversity.where(test_1, average_elevation)

Agora, nós adicionamos *wetland score*, primeiro testando onde *wetland score* é maior que a diversidade da paisagem calculada anteriormente, e nos locais cujos valores de *wetland score* são maiores que a diversidade da paisagem, nós substituímos os valores.


In [ ]:
# testando onde wetland score é maior que a diversidade da paisagem
# com variedade de landforms e amplitude de elevação
test_2 = wetland_score.gt(landscape_diversity)

# Médias para cada teste

average_wetland_1 = landform_Z.multiply(2).add(elevation_range_Z).add(wetland_score).divide(4)

average_wetland_2 = landform_Z.multiply(2).add(wetland_score).divide(3)

# Substitua o valor de diversidade da paisagem por wetland score
# onde o segundo é maior que o primeiro
landscape_diversity = landscape_diversity.where(test_1.And(test_2), average_wetland_1)

landscape_diversity = landscape_diversity.where(test_1.Not().And(test_2), average_wetland_2)

A inclusão da diversidade de solos é um pouco mais complexa. Nós testamos onde a diversidade de solo é maior a diversidade da paisagem já calculada e substituímos os valores por 4 médias ponderadas diferentes, cada uma representando os passos anteriores. A primeira média é para os locais onde a diversidade de solos é maior que *wetland score*, que é maior que amplitude de elevação. Na segunda média, os valores são para as regiões com maior diversidade de solos, mas onde *wetland score* é menor que a amplitude de elevação. A terceira média é para locais onde a amplitude de elevação não é maior que variedade de *landforms*, mas possui valores menores de *wetland score*, que também são menores que a diversidade de solos. A quarta média é para locais onde somente a diversidade de solos é maior que a variedade de *landforms*.


In [ ]:
# teste onde diversidade de solo é maior que a diversidade da paisagem
test_3 = soil_diversity.gt(landscape_diversity)

# Calcule as médias ponderadas
average_soil_1 = (landform_Z.multiply(2)
                 .add(elevation_range_Z)
                 .add(wetland_score)
                 .add(soil_diversity)
                 .divide(5))

average_soil_2 = (landform_Z.multiply(2)
                 .add(elevation_range_Z)
                 .add(soil_diversity)
                 .divide(4))

average_soil_3 = (landform_Z.multiply(2)
                 .add(wetland_score)
                 .add(soil_diversity)
                 .divide(4))

average_soil_4 = (landform_Z.multiply(2)
                 .add(soil_diversity)
                 .divide(3))

# Substitua os valores de diversidade da paisagem pela diversidade de solo
# os os valores de solo são maiores
landscape_diversity = (landscape_diversity
                      .where(test_1.And(test_2).And(test_3), average_soil_1))

landscape_diversity = (landscape_diversity
                      .where(test_1.And(test_2.Not()).And(test_3), average_soil_2))

landscape_diversity = (landscape_diversity
                      .where(test_1.Not().And(test_2).And(test_3), average_soil_3))

landscape_diversity = (landscape_diversity
                      .where(test_1.Not().And(test_2.Not()).And(test_3), average_soil_4))

 Finalizamos exportando o mapa de diversidade da paisagem como *asset* no *Google Earth Engine*.


In [ ]:
# Exporte o raster de diversidade da paisagem
assetId = "projects/ee-lucasljardim9/assets/landscape_diversity"

geemap.ee_export_image_to_asset(
    landscape_diversity, 
    description='landscape_diversity', 
    assetId=assetId, 
    region=bioma_box, 
    scale=escala,
    maxPixels=1e13
)